In [7]:
!pip install tensorflow
!pip install tensorflowjs
!pip install flask
!pip install flask_ngrok
!pip install ngrok
!pip install pyngrok
!pip install flask-cors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 38.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Succ

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=8a26250bf47c5a931e943237b0fe2525ba56251d6ba292955cafe79c6ce3dc9b
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import tensorflowjs as tfjs


In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


11490434/11490434 [==============================] - 0s 0us/step


In [10]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 13s 3ms/step - loss: 0.2519 - accuracy: 0.9288
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1136 - accuracy: 0.9663
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0791 - accuracy: 0.9752
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0584 - accuracy: 0.9818
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0452 - accuracy: 0.9861


In [12]:
model.save('mnist_model.h5')
tfjs.converters.save_keras_model(model, 'mnist_model_tfjs')
model.save('/content/drive/MyDrive/mnist_model.h5')



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!ngrok authtoken 'add yours '


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
from flask import Flask, request, jsonify
import numpy as np
from pyngrok import ngrok
import os
import io
import tensorflow as tf
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)


model = tf.keras.models.load_model('mnist_model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    # Check if the request contains a file with the key 'file'
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    # Check if the file is empty
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    # Check if the file is a PNG image
    if file.mimetype != 'image/png':
        return jsonify({'error': 'Unsupported file format. Only PNG images are supported.'})

    # Load and preprocess the image
    image = tf.keras.preprocessing.image.load_img(io.BytesIO(file.read()), target_size=(28, 28), color_mode='grayscale')
    input_data = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    input_data = np.expand_dims(input_data, axis=0)

    # Make prediction
    result = model.predict(input_data)
    predicted_class = np.argmax(result, axis=1)[0]

    return jsonify({'prediction': int(predicted_class)})

@app.route('/', methods=['GET'])
def hello():
    return 'Hello from server!'

if __name__ == '__main__':
   app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bbe0-104-154-30-217.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


In [15]:
from google.colab import files

files.download('/content/drive/MyDrive/mnist_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>